In [1]:
import sqlite3
from sqlite3 import Error
import unittest

path = 'A.db'

def create_connection(path, isolation_level=None):
    # connect to the database
    connection = None
    try:
        connection = sqlite3.connect(path, isolation_level=isolation_level)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

class TestMRP(unittest.TestCase):

    def setUp(self):
        self.connection = create_connection('A.db', isolation_level=None)
        self.cursor = self.connection.cursor()

    def test(self):
        # Insert or replace sample data into the database
        self.cursor.execute("INSERT OR REPLACE INTO MRP (part_id, period_id, demand, inventory, SR, GR, NR, POREC, POREL) VALUES (2, 6, 5, 0, 0, 0, 0, 0, 0)")

        self.connection.commit()

        # Perform MRP calculation and fetch results
        self.cursor.execute("SELECT part_id, component_part_id * multiplier as 'MRP' FROM BOM")
        results = self.cursor.fetchall()

        # Results
        print(results)

    def tearDown(self):
        # Close the connection to the database
        self.connection.close()

if __name__ == '__main__':
    # Create tables with unique constraints enforced
    create_period_table = """
    CREATE TABLE IF NOT EXISTS period (
      period_id INTEGER PRIMARY KEY,
      informatic INTEGER);
    """
    create_part_table = """
    CREATE TABLE IF NOT EXISTS part (
      part_id INTEGER PRIMARY KEY, 
      lead_time INTEGER, 
      initial_inventory INTEGER, 
      lot_size INTEGER, 
      level INTEGER);
    """
    create_BOM_table = """
    CREATE TABLE IF NOT EXISTS BOM (
      part_id INTEGER PRIMARY KEY, 
      component_part_id INTEGER, 
      multiplier INTEGER, 
      make_by_product INTEGER, 
      FOREIGN KEY (component_part_id) REFERENCES part (part_id));
    """
    create_MRP_table = """
    CREATE TABLE IF NOT EXISTS MRP (
      part_id INTEGER PRIMARY KEY, 
      period_id INTEGER, 
      demand INTEGER,
      inventory INTEGER,
      SR INTEGER,
      GR INTEGER,
      NR INTEGER,
      POREC INTEGER,
      POREL INTEGER,
      FOREIGN KEY (part_id) REFERENCES part (part_id),
      FOREIGN KEY (period_id) REFERENCES period (period_id));
    """

    # Execute table creation queries with unique constraints enforced
    connection = create_connection(path)
    execute_query(connection, create_period_table)
    execute_query(connection, create_part_table)
    execute_query(connection, create_BOM_table)
    execute_query(connection, create_MRP_table)
    connection.close()

    # Run tests with unique constraints temporarily ignored
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


Connection to SQLite DB successful
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Connection to SQLite DB successful
[]
